## Manim - Clase 3

In [ ]:
from manim import *

### Sincronización de animaciones

Algo que hasta ahora nos vino faltando es algún elemento que nos permita sincronizar animaciones, de modo de tener en pantalla varias cosas ocurriendo al mismo tiempo, coordinadas. 

Uno de los principales recursos para lograr esto proviene de la combinación de un objeto de clase `ValueTracker` con la función `always_redraw`. 

Un `ValueTracker` es esencialmente una variable numérica, con la particularidad de que nos permite informar a otros elementos de sus cambios de valor. Veamos un ejemplo:

### Primera Animación

Recomendamos correr el siguiente bloque y mirar la animación resultante antes de leer el código. Ayuda tener en mente el resultado, para comprender la lógica subyacente. 

In [ ]:
%%manim -qm C3A00

class C3A00(Scene):
    def construct(self):
        ax         = Axes(x_range=[-4, 4],
                          y_range=[-5, 5])
        a          = ValueTracker(1)
        f          = lambda x: a.get_value() * x**2
        func       = always_redraw(lambda: ax.plot(f, color=BLUE))
        
        formula_f  = MathTex("f(x)=ax^2").to_corner(UL)
        etiqueta_a = MathTex("a=")
        etiqueta_a.next_to(formula_f, DOWN).align_to(formula_f, LEFT, LEFT)
        valor_a    = always_redraw(lambda: MathTex(str('%.2f' % (a.get_value()))).next_to(etiqueta_a, RIGHT))

        self.play(Create(ax))
        self.play(Create(func))
        self.play(FadeIn(etiqueta_a), FadeIn(valor_a), FadeIn(formula_f))
        
        self.play(a.animate.set_value(2), runtime=2)
        self.wait()
        self.play(a.animate.set_value(1), runtime=2)
        self.play(a.animate.set_value(0.5), runtime=2)
        self.wait()
        self.play(a.animate.set_value(-1), runtime=2)
        self.wait()
        self.play(a.animate.set_value(-2), runtime=2)
        self.wait()

#### Análisis

El video hace algo de lo que nos proponemos: define una cuadrática de la forma $f(x)=ax^2$ y luego muestra el efecto de cambiar el parámetro $a$. Esto implica hacer dos cosas: por un lado corregir el dibujo y por el otro modificar el texto en el que se muestra el valor de $a$. 

¿Cómo se logra esto? 

Definimos `a = ValueTracker(1)`. Es decir que la variable `a` es un ValueTracker cuyo valor inicial es 1. Luego, definimos la función `f` utilizando como coeficiente `a.get_value()`. Esto extrae el valor del `ValueTracker` `a`. Por ahora ese valor es 1 y por lo tanto la función es $x^2$. Pero es importante tener en cuenta que ese 1 no está fijo, sino que proviene de tomar el valor de `a`, que podría cambiar. 

Luego, definimos el gráfico de la función usando `ax.plot`. Pero esta definición la hacemos dentro de `always_redraw`:

    func       = always_redraw(lambda: ax.plot(f, color=BLUE))

`always_redraw` debe recibir una función. Por eso escribimos `lambda:`. Esta función no tiene ningún parámetro, pero *devuelve* el gráfico de la función `f` (que a su vez depende del valor de `a`). `always_redraw` es una función propia de Manim que se corre en cada cuadro de la animación. Es decir que cada vez que Manim genere un cuadro del video va a *redibujar* lo que nosotros le pedimos: en este caso, el gráfico de `f`. Esto hace que el gráfico no sea una figura estática, sólo susceptible de ser movida, rotada o transformada a través de la aplicación explícita de alguna de las funciones de Manim. Por el contrario: el gráfico todo el tiempo es redibujado y puede cambiar si los parámetros que lo definen cambian. Si pasamos por alto la parte del texto y saltamos a las animaciones, podemos observar que fuera de la creación de los distintos elementos (ejes, gráfico, texto), las únicas animaciones son del tipo: 

    self.play(a.animate.set_value(2))

Es decir: no estamos animando explícitamente el gráfico de la función. Lo que estamos animando es el cambio de valor de `a`. `a` pasa de su valor original (1) a valer 2. Como el `set_value()` lo ponemos luego de `animate` este proceso de cambio es animado, es decir que `a` transita por un montón de valores intermedios para pasar de 1 a 2. Cada valor intermedio de `a` generará un nuevo cuadro en el video y ese cuadro, gracias al uso de `always_redraw` implicará la recreación del gráfico de `f`, que a su vez ha cambiado porque depende del (nuevo) valor de `a`. Si se observa el código se pueden ver 5 animaciones de cambios de valor de `a`:  de 1 a 2, de vuelta a 1, a 0.5, a -1 y a -2. En el video pueden verse pequeñas pausas luego de cada movimiento de `a`. 

Si ahora volvemos atrás a mirar el texto, podemos encontrar un par de elementos conocidos: definimos `formula_f` usando `MathTex` y la posicionamos en la esquina superior izquierda; definimos `label_a` cuyo texto (estático) es `a = ` y la posicionamos debajo de `formula_a` y alineada en el borde izquierdo. Por último, definimos `valor_a` que es el texto cambiante en el video. Para esto, usamos nuevamente `always_redraw`, sólo que el objeto que queremos crear no es un `plot` sino algo de tipo `MathTex`. El contenido de ese objeto de tipo `MathTex` es:

    str('%.2f' %(a.get_value()))

Esto es simplemente un tecnicismo (útil, pero tecnicismo al fin). No podemos poner `MathTex(a.get_value())`, porque `a.get_value()` devuelve un número (de tipo `float`) y no texto y por lo tanto `MathTex` no es capaz de entender esa sintaxis. Una alternativa es escribir `str(a.get_value())` que convierte el valor de `a` en un *string* (cadena de caracteres). El problema es que `a` puede ser un número con muchísimos decimales que **no** queremos ver en pantalla. La sintaxis que usamos simplemente le indica a `str` que debe tomar el valor definido luego de `%` y traducirlo a texto, con punto decimal y dos dígitos detrás de ese punto. La `f` viene de `float` (número flotante). Pueden jugar con esta configuración. Por ejemplo:  probar `str(a.get_value))`, cambiar el número de decimales, etc. 

De esta manera logramos *sincronizar* dos animaciones distintas: la del gráfico y la del texto. Pero en realidad, no sincronizamos dos animaciones: sólo hicimos **una** animación, que es la del cambio de valor de `a`. Lo que ocurre es que tanto el texto como el gráfico dependen de `a` y se actualizan gracias a `always_redraw`. 

### Segunda Animación

Siguiendo la misma lógica podemos animar, por ejemplo, un punto moviéndose a lo largo del gráfico de una función. 

In [ ]:
%%manim -qm C3A01

class C3A01(Scene):
    def construct(self):
        ax = Axes(x_range=[-3, 3],
                  y_range=[-0.5, 3],
                  x_length=8,
                  y_length=5)
        labels = ax.get_axis_labels(x_label="x", y_label="f(x)")
        labels[0].scale(0.5)
        labels[1].scale(0.5)
        
        def f(x): return 2*np.exp(-x**2)
        grafico = ax.plot(f, color=BLUE)
        
        x0      = -1
        t   = ValueTracker(x0)
        dot = always_redraw(lambda : Dot(ax.c2p(t.get_value(), f(t.get_value())), color=RED))
        
        self.play(Create(ax), Create(labels))
        self.wait()
        self.play(Create(grafico), runtime=2)
        self.wait()
        self.play(FadeIn(dot))
        self.wait()
        self.play(t.animate.set_value(1), runtime=2)
        self.wait()

#### Análisis

Creamos un par de ejes y le agregamos sus correspondientes etiquetas. A estas etiquetas les aplicamos la función `scale` (propia de todo Mobject) que nos permite cambiarle el tamaño *in situ*. En este caso las achicamos a la mitad de su tamaño estandar. 

Lo interesante viene después. Definimos un `ValueTracker`, `t`, 